In [0]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [0]:
#fix random seed reproductivity
seed = 7
np.random.seed(seed)

In [0]:
import io
from google.colab import files
uploaded = files.upload()
df = pd.read_csv(io.StringIO(uploaded['sonar.all-data'].decode('utf-8')))
df

Saving sonar.all-data to sonar.all-data


,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
0,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
1,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
2,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
3,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
4,0.0286,0.0453,0.0277,0.0174,0.0384,0.0990,0.1201,0.1833,0.2105,0.3039,0.2988,0.4250,0.6343,0.8198,1.0000,0.9988,0.9508,0.9025,0.7234,0.5122,0.2074,0.3985,0.5890,0.2872,0.2043,0.5782,0.5389,0.3750,0.3411,0.5067,0.5580,0.4778,0.3299,0.2198,0.1407,0.2856,0.3807,0.4158,0.4054,0.3296,0.2707,0.2650,0.0723,0.1238,0.1192,0.1089,0.0623,0.0494,0.0264,0.0081,0.0104,0.0045,0.0014,0.0038,0.0013,0.0089,0.0057,0.0027,0.0051,0.0062,R
5,0.0317,0.0956,0.1321,0.1408,0.1674,0.1710,0.0731,0.1401,0.2083,0.3513,0.1786,0.0658,0.0513,0.3752,0.5419,0.5440,0.5150,0.4262,0.2024,0.4233,0.7723,0.9735,0.9390,0.5559,0.5268,0.6826,0.5713,0.5429,0.2177,0.2149,0.5811,0.6323,0.2965,0.1873,0.2969,0.5163,0.6153,0.4283,0.5479,0.6133,0.5017,0.2377,0.1957,0.1749,0.1304,0.0597,0.1124,0.1047,0.0507,0.0159,0.0195,0.0201,0.0248,0.0131,0.0070,0.0138,0.0092,0.0143,0.0036,0.0103,R
6,0.0519,0.0548,0.0842,0.0319,0.1158,0.0922,0.1027,0.0613,0.1465,0.2838,0.2802,0.3086,0.2657,0.3801,0.5626,0.4376,0.2617,0.1199,0.6676,0.9402,0.7832,0.5352,0.6809,0.9174,0.7613,0.8220,0.8872,0.6091,0.2967,0.1103,0.1318,0.0624,0.0990,0.4006,0.3666,0.1050,0.1915,0.3930,0.4288,0.2546,0.1151,0.2196,0.1879,0.1437,0.2146,0.2360,0.1125,0.0254,0.0285,0.0178,0.0052,0.0081,0.0120,0.0045,0.0121,0.0097,0.0085,0.0047,0.0048,0.0053,R
7,0.0223,0.0375,0.0484,0.0475,0.0647,0.0591,0.0753,0.0098,0.0684,0.1487,0.1156,0.1654,0.3833,0.3598,0.1713,0.1136,0.0349,0.3796,0.7401,0.9925,0.9802,0.8890,0.6712,0.4286,0.3374,0.7366,0.9611,0.7353,0.4856,0.1594,0.3007,0.4096,0.3170,0.3305,0.3408,0.2186,0.2463,0.2726,0.1680,0.2792,0.2558,0.1740,0.2121,0.1099,0.0985,0.1271,0.1459,0.1164,0.0777,0.0439,0.0061,0.0145,0.0128,0.0145,0.0058,0.0049,0.0065,0.0093,0.0059,0.0022,R
8,0.0164,0.0173,0.0347,0.0070,0.0187,0.0671,0.1056,0.0697,0.0962,0.0251,0.0801,0.1056,0.1266,0.0890,0.0198,0.1133,0.2826,0.3234,0.3238,0.4333,0.6068,0.7652,0.9203,0.9719,0.9207,0.7545,0

In [0]:
df.head()

,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
0,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
1,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
2,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
3,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
4,0.0286,0.0453,0.0277,0.0174,0.0384,0.0990,0.1201,0.1833,0.2105,0.3039,0.2988,0.4250,0.6343,0.8198,1.0000,0.9988,0.9508,0.9025,0.7234,0.5122,0.2074,0.3985,0.5890,0.2872,0.2043,0.5782,0.5389,0.3750,0.3411,0.5067,0.5580,0.4778,0.3299,0.2198,0.1407,0.2856,0.3807,0.4158,0.4054,0.3296,0.2707,0.2650,0.0723,0.1238,0.1192,0.1089,0.0623,0.0494,0.0264,0.0081,0.0104,0.0045,0.0014,0.0038,0.0013,0.0089,0.0057,0.0027,0.0051,0.0062,R


In [0]:
dataset = df.values
print(dataset[0])
print(dataset.ndim)
print(dataset.shape)
print(type(dataset))

[0.0453 0.0523 0.0843 0.0689 0.1183 0.2583 0.2156 0.3481 0.3337 0.2872
 0.4918 0.6552 0.6919 0.7797 0.7464 0.9444 1.0 0.8874 0.8024 0.7818 0.5212
 0.4052 0.3957 0.3914 0.325 0.32 0.3271 0.2767 0.4423 0.2028 0.3788 0.2947
 0.1984 0.2341 0.1306 0.4182 0.3835 0.1057 0.184 0.197 0.1674 0.0583
 0.1401 0.1628 0.0621 0.0203 0.053 0.0742 0.0409 0.0061 0.0125 0.0084
 0.0089 0.0048 0.0094 0.0191 0.014 0.0049 0.0052 0.0044 'R']
2
(207, 61)
<class 'numpy.ndarray'>


In [0]:
x = dataset[:, 0:60].astype(float)
y= dataset[:,60]
print(x[0])
print(y)
print(x.shape)
print(y.shape)

[0.0453 0.0523 0.0843 0.0689 0.1183 0.2583 0.2156 0.3481 0.3337 0.2872
 0.4918 0.6552 0.6919 0.7797 0.7464 0.9444 1.     0.8874 0.8024 0.7818
 0.5212 0.4052 0.3957 0.3914 0.325  0.32   0.3271 0.2767 0.4423 0.2028
 0.3788 0.2947 0.1984 0.2341 0.1306 0.4182 0.3835 0.1057 0.184  0.197
 0.1674 0.0583 0.1401 0.1628 0.0621 0.0203 0.053  0.0742 0.0409 0.0061
 0.0125 0.0084 0.0089 0.0048 0.0094 0.0191 0.014  0.0049 0.0052 0.0044]
['R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M'
 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M' 'M

In [0]:
LE = LabelEncoder()
LE.fit(y)
y_encode = LE.transform(y)
y_encode

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])

In [0]:
#To create a baseline
from keras import optimizers
def create_baseline():
  network = Sequential()
  network.add(Dense(60, activation = 'relu' , input_shape = (60,)))
  network.add(Dense(1, activation ='sigmoid'))
  network.compile(optimizer = 'rmsprop' , loss = 'binary_crossentropy' , metrics=['accuracy'])
  return network

In [0]:
#evaluate
estimator = KerasClassifier(build_fn = create_baseline , epochs = 100 , batch_size = 5 , verbose =0)
kfold = StratifiedKFold(n_splits = 10 , shuffle = True , random_state = seed)
results = cross_val_score(estimator , x , y_encode , cv= kfold)
print("Results : %2f%%(%2f%%)" %(results.mean()*100, results.std()*100))


W0827 07:50:52.221846 140303861704576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0827 07:50:52.240642 140303861704576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0827 07:50:52.247563 140303861704576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0827 07:50:52.277106 140303861704576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0827 07:50:52.297330 140303861704576 deprecation_wrappe

Results : 79.705629%(6.046413%)


In [0]:
#Evaluate Baseline model
np.random.seed(seed)
estimators =[]
estimators.append(('standarize' , StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn = create_baseline , epochs=100 , batch_size =5 , verbose = 0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits = 10, shuffle= True , random_state= seed)
results = cross_val_score(pipeline , x , y_encode , cv= kfold)
print("Standarized : %2f%%(%.2f%%)" %(results.mean()*100 , results.std()*100))

Standarized : 83.062771%(6.58%)


In [0]:
#TO EVALUATE THE SMALLER NETWORK

def create_smaller():
  small_network = Sequential()
  small_network.add(Dense(30, activation= 'relu',input_shape=(60,)))
  small_network.add(Dense(1, activation = 'sigmoid'))
  small_network.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy' , metrics =['acc'])
  return small_network

In [0]:
#Run
estimators =[]
estimators.append(('standarize' , StandardScaler()))
estimators.append(('mlp' , KerasClassifier(build_fn = create_smaller , epochs =100 , batch_size = 5 , verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits =10 , shuffle = True , random_state = seed)
results = cross_val_score(pipeline , x , y_encode , cv= kfold)
print('smaller: %2f%%(%2f%%)' % (results.mean()*100 , results.std()*100))


smaller: 83.112555%(5.700074%)


In [0]:
#TO CREATE A larger network
from keras import optimizers 
def create_larger():
  large_network = Sequential()
  large_network.add(Dense(60, activation = 'relu' , input_shape =(60,)))
  large_network.add(Dense(30, activation='relu'))
  large_network.add(Dense(1, activation='sigmoid'))
  large_network.compile(optimizer = 'rmsprop' , loss = 'binary_crossentropy' , metrics=['acc'])
  return large_network

In [0]:
#RUn 
estimators = []
estimators.append(('standarize' , StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn = create_larger , epochs = 100 , batch_size = 5 , verbose =0 )))
pipeline = Pipeline(estimators)
kfold= StratifiedKFold(n_splits = 10 , shuffle = True , random_state= seed)
results = cross_val_score(pipeline , x , y_encode , cv= kfold)
print("larger : %2f%% (%2f%%)" % (results.mean()*100 , results.std()*100))

larger : 81.564936% (6.211201%)


In [0]:
#Create baseline 
from keras import optimizers 
from keras import regularizers
def create_final():
  final_network = Sequential()
  final_network.add(Dense(30, activation= 'relu' , input_shape=(60,)))
  final_network.add(Dense(1, activation='sigmoid'))
  final_network.compile(optimizer = 'rmsprop' , loss= 'binary_crossentropy' , metrics=['acc'])
  return final_network

In [0]:
#Evaluate Model

estimator = KerasClassifier(build_fn = create_final , epochs=100, batch_size = 5 , verbose=0)
kfold= StratifiedKFold(n_splits= 10 , shuffle=True , random_state=seed)
results = cross_val_score(estimator , x , y_encode , cv = kfold)
print("Results : %2f%%(%2f%%)" %(results.mean()*100 , results.std()*100))


Results : 80.729438%(6.359174%)


In [0]:
#Functional API
from keras.layers import Input , Dense
from keras.models import Model
def functional_model():
  inputs = Input(shape=(60,))
  z = Dense(30, activation='relu')(inputs)
  outputs= Dense(1, activation='sigmoid')(z)
  model= Model(inputs, outputs)
  model.compile(optimizer = 'Adam', loss='binary_crossentropy' , metrics=['acc'])
  return model

In [0]:
#Run the model
estimators=[]
estimators.append(('standarize' , StandardScaler()))
estimators.append(('mlp' , KerasClassifier(build_fn= functional_model , epochs = 100 , batch_size=5 , verbose=0)))
pipeline = Pipeline(estimators)
kfold= StratifiedKFold(n_splits= 10 , shuffle=True , random_state= seed)
results= cross_val_score(pipeline, x , y_encode , cv= kfold)
print('funcational api: %2f%%(%2f%%)'%(results.mean()*100 , results.std()*100))

funcational api: 84.015152%(6.891404%)


In [0]:
#Build Model with Model subclassing 
import tensorflow as tf
class SubclassModel(tf.keras.Model):
  def __init__(self):
    super(SubclassModel, self).__init__()
    self.dense1= tf.keras.layers.Dense(60, activation= tf.nn.relu)
    self.dense2= tf.keras.layers.Dense(30, activation= tf.nn.relu)
    self.dense3= tf.keras.layers.Dense(1, activation= tf.nn.sigmoid)
  def call(self,inputs):
    x = self.dense1(inputs)
    x= self.dense2(x)
    return self.dense3(x)
def finalModel():
  model = SubclassModel()
  model.compile(loss= 'binary_crossentropy' , optimizer = 'adam' , metrics= ['acc'])
  return model
    
    

In [0]:
estimators =[]
estimators.append(('standarize' , StandardScaler()))
estimators.append(('mlp' , KerasClassifier(build_fn= finalModel , epochs = 100 , batch_size= 5 , verbose=0)))
pipeline= Pipeline(estimators)
kfold= StratifiedKFold(n_splits = 10 , shuffle= True , random_state= seed)
results = cross_val_score(pipeline , x , y_encode, cv=kfold) 
print('Subclass larger Model : %2f%%(%2f%%)'% (results.mean()*100, results.std()*100))


W0827 08:22:07.439453 140303861704576 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Subclass larger Model : 83.541126%(6.258041%)


In [0]:
#without scikit learn

print(x)
print(y_encode)


[[0.0453 0.0523 0.0843 ... 0.0049 0.0052 0.0044]
 [0.0262 0.0582 0.1099 ... 0.0164 0.0095 0.0078]
 [0.01   0.0171 0.0623 ... 0.0044 0.004  0.0117]
 ...
 [0.0522 0.0437 0.018  ... 0.0138 0.0077 0.0031]
 [0.0303 0.0353 0.049  ... 0.0079 0.0036 0.0048]
 [0.026  0.0363 0.0136 ... 0.0036 0.0061 0.0115]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [0]:
complete_data = dataset.copy()
np.random.shuffle(complete_data)
data = complete_data[:,0:60].astype('float32')
labels= complete_data[:,60]
le = LabelEncoder()
le.fit(labels)
labels_encode= le.transform(labels).astype('float32')
print(labels_encode)
print(data.dtype)
train_data= data[:140]
test_data=data[140:]
train_labels= labels_encode[:140]
test_labels= labels_encode[140:]
print(train_data.shape)
print(test_data.shape)
print(train_labels.shape)
print(test_labels.shape)
print(train_labels[0])


[1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0.
 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0.
 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0.
 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0.
 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1.
 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0.
 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0.
 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1.]
float32
(140, 60)
(67, 60)
(140,)
(67,)
1.0


In [0]:
def build_model():
  model= Sequential()
  model.add(Dense(60, activation='relu' , input_shape=(60,)))
  model.add(Dense(30, activation='relu'))
  model.add(Dense(1 , activation='sigmoid'))
  model.compile(optimizer='Adam' , loss= 'binary_crossentropy' , metrics=['acc'])
  return model

In [0]:
k=4
num_val_samples= len(train_data)//k
num_epochs= 100

for i in range(k):
  print('processing fold#' , i)
  val_data = train_data[i* num_val_samples: (i+1) * num_val_samples]
  val_labels = train_labels[i* num_val_samples : (i+1) * num_val_samples]
  partial_train_data= np.concatenate([train_data[:i* num_val_samples], train_data[(i+1)*num_val_samples:]], axis=0)
  partial_train_labels = np.concatenate([train_labels[:i*num_val_samples], train_labels[(i+1)*num_val_samples:]], axis=0)
  
  model= build_model()
  model.fit(partial_train_data, partial_train_labels , epochs= num_epochs , batch_size=5 , verbose= 0)
  loss , acc = model.evaluate(val_data , val_labels, verbose=0)
  print(acc)

processing fold# 0
0.8857142865657807
processing fold# 1
0.7714285722800663
processing fold# 2
0.7714285714285715
processing fold# 3
0.6285714302744184


In [0]:
def last_model():
  model = Sequential()
  model.add(Dense(60 , activation='relu' , input_shape=(60,)))
  model.add(Dense(30, activation='relu'))
  model.add(Dense(1 , activation='sigmoid'))
  model.compile(optimizer='Adam' , loss='binary_crossentropy' , metrics=['acc'])
  return model

model= last_model()
model.fit(train_data , train_labels , epochs= 100 , batch_size= 5 , verbose=0)

In [0]:
history= model.evaluate(test_data , test_labels) 
history

67/67 [==============================] - 2s 23ms/step


[0.44837546901805186, 0.8656716417910447]